Thêm Thư viện

In [11]:
import math
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor

Load data từ file BikeSharingDemand

In [ ]:
# Load dữ liệu từ file CSV
dt = pd.read_csv("BikeSharingDemand.csv")
print ( "Hiển thị 5 dòng đầu của tập data\n ", dt.head())
print ( "Xem kiểu dữ liệu của từng thuộc tính ", dt.dtypes)
print ( "Xem thông tin của dữ liệu ", dt.info())
print ( "Kiểm tra giá trị thiếu ",dt.isnull().sum())

Chú thích cho tập dữ liệu  <br>
[!FAQ]
||datetime: cho biết thời gian theo dịnh dạng "%m/%d/%Y %H:%M:%hS  <br> 
season: cho biết mùa  <br> 
1: mùa xuân  <br>
2: mùa hè  <br>
3: mùa thu  <br>
4: mùa đông  <br>
workingday: cho biết có phải là ngày làm việc không bao gồm cả cuối tuần và ngày lễ  <br> 
1: ngày làm việc  
0: ngày không làm việc 
weather: Cho biết thời tiết  <br>
1: Trời trong, có mây, có mây rải rác  <br>
2: Sương mù + Đầy mây, sương mù + ít mây, sương mù  <br>
3: Tuyết nhẹ, mưa nhẹ + sấm sét + mây rải rác, mưa nhẹ + mây rải rác  <br>
4: Mưa lớn + mảnh băng + sấm sét + sương mù, tuyết + sương mù  <br>
temp: Nhiệt độ thực tế tính theo độ Celsius (độ C)  <br>
atemp: Nhiệt độ mà con người cảm thấy tính theo độ Celsius (độ C) <br>
humidity: Độ ẩm tương đối <br>
windspeed: Tốc độ gió <br>
casual: Số lượng xe thuê mà không có đăng ký dịch vụ <br>
registered: Số lượng xe thuê mà có đăng ký dịch vụ <br>
count: Tổng số lượng xe được thuê <br>

Xử lý Data

In [ ]:
# tạo cột year, month, day, hour,weekday từ cột datetime
dt["datetime"] = pd.to_datetime(dt["datetime"])
dt["year"] = dt["datetime"].dt.year
dt["month"] = dt["datetime"].dt.month
dt["weekday"] = dt["datetime"].dt.weekday
dt["hour"] = dt["datetime"].dt.hour
print(" Hiển thị 5 dòng đầu của tập data\n",  dt.head())
print(" Xem kiểu dữ liệu của từng thuộc tính",  dt.dtypes)
print(" Xem thông tin của dữ liệu",  dt.info())
print(" Kiểm tra giá trị thiếu", dt.isnull().sum())

Chú thích các thuộc tính mới  <br>
month: tháng (1-12)  <br>
year: năm (2011, 2012)  <br>
hour: giờ trong ngày (0-23 giờ)  <br>
weekday: ngày trong tuần  <br> 
0: thứ 2  <br>
1: thứ 3  <br>
2: thứ 4  <br>
3: thứ 5  <br>
4: thứ 6  <br>
5: thứ 7  <br>
6: chủ nhật  <br>

Chia Dữ tập dữ liệu theo Hold-Out <br>
Test = 1/3 , train = 2/3  <br>

In [ ]:
# Chia dữ liệu thành X và y
X = dt.drop(columns=["casual", "registered", "count", "datetime"])
print(X)
y = dt["count"] 
print(y)
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1 / 3, random_state=42
)
print(X_train)
print(X_test)

Xây dụng mô hình bằng phương pháp tổng hợp mô hình (Bagging Average) <br>
Với mô hình cơ sở là DecisionTreeRegressor (cây hồi quy)  <br>

In [ ]:
tree = DecisionTreeRegressor()
# Tạo 1 tập hợp gồm 10 mô hình cây hồi quy
baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
# huấn luyện mô hình 
baggingTree.fit(X_train, y_train)
# Dự đoán kết quả
y_pred = baggingTree.predict(X_test)
# Đánh giá mô hình
err_DT = mean_squared_error(y_test, y_pred)
print("DecisionTreeRegressor MSE =", str(err_DT))
rmse_err_DT = math.sqrt(err_DT)
print("DecisionTreeRegressor RMSE =" + str(round(rmse_err_DT, 3)))

Huấn luyện mô hình

Xây dụng mô hình bằng phương pháp tổng hợp mô hình (Bagging Average)
Với mô hình cơ sở là LinearRegression (Hồi quy tuyến tính) 

In [ ]:
lm = linear_model.LinearRegression()
# Tạo 1 tập hợp gồm 10 mô hình hồi quy tuyến tính
baggingLM = BaggingRegressor(estimator=lm, n_estimators=10, random_state=42)
# huấn luyện mô hình
baggingLM.fit(X_train, y_train)
# dụ đoán kết quả
y_pred = baggingLM.predict(X_test)
# đánh giá mô hình 
err_LM = mean_squared_error(y_test, y_pred)
print("LinearRegression MSE =", str(err_LM))
rmse_err_LM = math.sqrt(err_LM)
print("LinearRegression RMSE =" + str(round(rmse_err_LM, 3)))

Đánh giá từng mô hình

In [ ]:
# Tạo mô hình cây hồi quy
treeRegressor = DecisionTreeRegressor()
treeRegressor.fit(X_train, y_train)
# Dự đoán kết quả
y_pred_tree = treeRegressor.predict(X_test)
# đánh giá mô hình
err_tree = mean_squared_error(y_test, y_pred_tree)
print("MSE of tree = " + str(err_tree))
rmse_err_tree = math.sqrt(err_tree)
print("RMSE of tree = " + str(round(rmse_err_tree, 3)))

In [ ]:
# Tạo mô hình hồi quy tuyến tính
LmRegressor = LinearRegression()
LmRegressor.fit(X_train, y_train)
# Dự đoán kết quả
y_pred_lm = LmRegressor.predict(X_test)
# Đánh giả mô hình
err_lm = mean_squared_error(y_test, y_pred_lm)
print("MSE of lm = " + str(err_lm))
rmse_err_lm = math.sqrt(err_lm)
print("RMSE of lm = " + str(round(rmse_err_lm, 3)))

In [ ]:
# Tạo mô hình KNN
knnRegressor = KNeighborsRegressor(n_neighbors=10)
knnRegressor.fit(X_train, y_train)
# Dự đoán kết quả
y_pred_knn = knnRegressor.predict(X_test)
# Đánh Giá mô hình
err_knn = mean_squared_error(y_test, y_pred_knn)
print("MSE of KNN = " + str(err_knn))
rmse_err_knn = math.sqrt(err_knn)
print("RMSE of KNN = " + str(round(rmse_err_knn, 3)))


Kết hợp 3 mô hình 

In [ ]:
# Kết hợp mô hình với VotingRegressor
voting_reg = VotingRegressor(
    estimators=[
        ("tree_reg", treeRegressor),
        ("Lm_reg", LmRegressor),
        ("knn_reg", knnRegressor),
    ]
)
voting_reg.fit(X_train, y_train)
# Dự Đoán kết quả
y_pred_ensemble = voting_reg.predict(X_test)
# Đánh giá mô hình
err_ensemble = mean_squared_error(y_test, y_pred_ensemble)
print("MSE of Ensemble = " + str(err_ensemble))
rmse_err_ensemble = math.sqrt(err_ensemble)
print("RMSE of Ensemble = " + str(round(rmse_err_ensemble, 3)))

Chia tập dữ liệu theo Kfold

Load Data

In [ ]:
# Load dữ liệu từ file CSV
dt = pd.read_csv("./BikeSharingDemand.csv")
print(dt)
print(dt.dtypes)
print(dt.info)

Xử lý data

In [13]:
# tạo cột year, month, day, hour,weekday từ cột datetime
dt["datetime"] = pd.to_datetime(dt["datetime"])
dt["year"] = dt["datetime"].dt.year
dt["month"] = dt["datetime"].dt.month
dt["weekday"] = dt["datetime"].dt.weekday
dt["hour"] = dt["datetime"].dt.hour
print(dt)
print(dt.dtypes)
print(dt.info)
print(dt.describe())

                 datetime  season  holiday  workingday  weather   temp  \
0     2011-01-01 00:00:00       1        0           0        1   9.84   
1     2011-01-01 01:00:00       1        0           0        1   9.02   
2     2011-01-01 02:00:00       1        0           0        1   9.02   
3     2011-01-01 03:00:00       1        0           0        1   9.84   
4     2011-01-01 04:00:00       1        0           0        1   9.84   
...                   ...     ...      ...         ...      ...    ...   
10881 2012-12-19 19:00:00       4        0           1        1  15.58   
10882 2012-12-19 20:00:00       4        0           1        1  14.76   
10883 2012-12-19 21:00:00       4        0           1        1  13.94   
10884 2012-12-19 22:00:00       4        0           1        1  13.94   
10885 2012-12-19 23:00:00       4        0           1        1  13.12   

        atemp  humidity  windspeed  casual  registered  count  year  month  \
0      14.395        81     0.000

Chia tập dữ liệu theo kfold 

In [14]:
# Chia dữ liệu thành X và y
X = dt.drop(columns=["casual", "registered", "count", "datetime"])
print(X)
y = dt["count"] 
print(y)
# Số lượng fold
n_splits = 50

# Khởi tạo KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

       season  holiday  workingday  weather   temp   atemp  humidity  \
0           1        0           0        1   9.84  14.395        81   
1           1        0           0        1   9.02  13.635        80   
2           1        0           0        1   9.02  13.635        80   
3           1        0           0        1   9.84  14.395        75   
4           1        0           0        1   9.84  14.395        75   
...       ...      ...         ...      ...    ...     ...       ...   
10881       4        0           1        1  15.58  19.695        50   
10882       4        0           1        1  14.76  17.425        57   
10883       4        0           1        1  13.94  15.910        61   
10884       4        0           1        1  13.94  17.425        61   
10885       4        0           1        1  13.12  16.665        66   

       windspeed  year  month  weekday  hour  
0         0.0000  2011      1        5     0  
1         0.0000  2011      1        5   

Xây dựng mô hình cây hồi quy

In [15]:


# Danh sách để lưu trữ MSE và RMSE cho từng mô hình
mse_results = []
rmse_results = []

# Lặp qua các fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    ## Bagging
    # Mô hình cơ sở là Decision Tree
    tree = DecisionTreeRegressor()
    # Tạo mô hình Bagging với 10 mô hình cơ sở
    baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
    # Huấn luyện mô hình
    baggingTree.fit(X_train, y_train)
    # Dự đoán kết quả
    y_pred = baggingTree.predict(X_test)
    # Đánh giá mô hình
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    mse_results.append(mse)
    rmse_results.append(rmse)

# Tính trung bình của MSE và RMSE từ các fold
avg_mse = sum(mse_results) / len(mse_results)
avg_rmse = sum(rmse_results) / len(rmse_results)

# In ra kết quả trung bình của MSE và RMSE
print("DecisionTree RegressorAverage MSE =", avg_mse)
print("DecisionTree Regressor Average RMSE =", avg_rmse)


DecisionTree RegressorAverage MSE = 1709.698860820445
DecisionTree Regressor Average RMSE = 40.961000641050894


Chia tập dữ liệu theo kfold và huấn luyện mô hình với hồi quy tuyến tính

In [17]:

# Danh sách để lưu trữ MSE và RMSE cho từng mô hình
mse_results = []
rmse_results = []

# Lặp qua các fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    ## Bagging
    # Mô hình cơ sở là Decision Tree
    tree = LinearRegression()
    # Tạo mô hình Bagging với 10 mô hình cơ sở
    baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
    # Huấn luyện mô hình
    baggingTree.fit(X_train, y_train)
    # Dự đoán kết quả
    y_pred = baggingTree.predict(X_test)
    # Đánh giá mô hình
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    mse_results.append(mse)
    rmse_results.append(rmse)

# Tính trung bình của MSE và RMSE từ các fold
avg_mse = sum(mse_results) / len(mse_results)
avg_rmse = sum(rmse_results) / len(rmse_results)

# In ra kết quả trung bình của MSE và RMSE
print("LinearRegressionAverage MSE =", avg_mse)
print("LinearRegression Average RMSE =", avg_rmse)


LinearRegressionAverage MSE = 20077.295855819408
LinearRegression Average RMSE = 141.45659600081123


Chia tập dữ liệu theo kfold và huấn luyện mô hình với KNN

In [18]:

# Danh sách để lưu trữ MSE và RMSE cho từng mô hình
mse_results = []
rmse_results = []

# Lặp qua các fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    ## Bagging
    # Mô hình cơ sở là Decision Tree
    tree = KNeighborsRegressor()
    # Tạo mô hình Bagging với 10 mô hình cơ sở
    baggingTree = BaggingRegressor(estimator=tree, n_estimators=10, random_state=42)
    # Huấn luyện mô hình
    baggingTree.fit(X_train, y_train)
    # Dự đoán kết quả
    y_pred = baggingTree.predict(X_test)
    # Đánh giá mô hình
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    mse_results.append(mse)
    rmse_results.append(rmse)

# Tính trung bình của MSE và RMSE từ các fold
avg_mse = sum(mse_results) / len(mse_results)
avg_rmse = sum(rmse_results) / len(rmse_results)

# In ra kết quả trung bình của MSE và RMSE
print("KNeighborsRegressor MSE =", avg_mse)
print("KNeighborsRegressor Average RMSE =", avg_rmse)


KNeighborsRegressor MSE = 12712.776250027651
KNeighborsRegressor Average RMSE = 112.36396651137751
